In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ''

In [34]:
from enum import Enum
from typing import Union

from pydantic import BaseModel

import openai
from openai import OpenAI


class Table(str, Enum):
    orders = "orders"
    customers = "customers"
    products = "products"


class Column(str, Enum):
    id = "id"
    status = "status"
    expected_delivery_date = "expected_delivery_date"
    delivered_at = "delivered_at"
    shipped_at = "shipped_at"
    ordered_at = "ordered_at"
    canceled_at = "canceled_at"


class Operator(str, Enum):
    eq = "="
    gt = ">"
    lt = "<"
    le = "<="
    ge = ">="
    ne = "!="


class OrderBy(str, Enum):
    asc = "asc"
    desc = "desc"


class DynamicValue(BaseModel):
    column_name: str


class Condition(BaseModel):
    column: str
    operator: Operator
    value: Union[str, int, DynamicValue]


class Query(BaseModel):
    table_name: Table
    columns: list[Column]
    conditions: list[Condition]
    order_by: OrderBy


client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. The current date is August 6, 2024. You help users query for the data they are looking for by calling the query function.",
        },
        {
            "role": "user",
            "content": "look up all my orders in may of last year that were fulfilled but not delivered on time",
        },
    ],
    tools=[
        openai.pydantic_function_tool(Query),
    ],
)

res = completion.choices[0].message.tool_calls[0].function.parsed_arguments
print(res.model_dump_json(indent=4))

{
    "table_name": "orders",
    "columns": [
        "id",
        "status",
        "expected_delivery_date",
        "delivered_at"
    ],
    "conditions": [
        {
            "column": "status",
            "operator": "=",
            "value": "fulfilled"
        },
        {
            "column": "ordered_at",
            "operator": ">=",
            "value": "2023-05-01"
        },
        {
            "column": "ordered_at",
            "operator": "<=",
            "value": "2023-05-31"
        },
        {
            "column": "delivered_at",
            "operator": ">",
            "value": {
                "column_name": "expected_delivery_date"
            }
        }
    ],
    "order_by": "asc"
}


In [35]:
from pydantic import BaseModel

from openai import OpenAI


class Step(BaseModel):
    explanation: str
    output: str


class MathResponse(BaseModel):
    steps: list[Step]
    final_answer: str


client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor."},
        {"role": "user", "content": "solve 8x + 31 = 2"},
    ],
    response_format=MathResponse,
)

message = completion.choices[0].message
if message.parsed:
    print(message.parsed.steps)
    print(message.parsed.final_answer)
else:
    print(message.refusal)

[Step(explanation='Start with the original equation: \\( 8x + 31 = 2 \\). To isolate the term with \\( x \\), we first subtract 31 from both sides of the equation.', output='8x + 31 - 31 = 2 - 31'), Step(explanation='This simplifies to \\( 8x = -29 \\). We now have the term with \\( x \\) isolated on one side of the equation.', output='8x = -29'), Step(explanation='To solve for \\( x \\), divide both sides of the equation by 8.', output='x = \\frac{-29}{8}')]
x = -\frac{29}{8}


In [40]:
print(message.parsed.model_dump_json(indent=4))

{
    "steps": [
        {
            "explanation": "Start with the original equation: \\( 8x + 31 = 2 \\). To isolate the term with \\( x \\), we first subtract 31 from both sides of the equation.",
            "output": "8x + 31 - 31 = 2 - 31"
        },
        {
            "explanation": "This simplifies to \\( 8x = -29 \\). We now have the term with \\( x \\) isolated on one side of the equation.",
            "output": "8x = -29"
        },
        {
            "explanation": "To solve for \\( x \\), divide both sides of the equation by 8.",
            "output": "x = \\frac{-29}{8}"
        }
    ],
    "final_answer": "x = -\\frac{29}{8}"
}


In [126]:

client = OpenAI()



completion = client.beta.chat.completions.parse(
      model= "gpt-4o-2024-08-06",
      messages= [
        {
          "role": "system",
          "content": "You are a helpful assistant"
        },
        {
          "role": "user",
          "content": "9.11 and 9.9 -- which is bigger?"
        }
      ],
      response_format = {
        "type": "json_schema",
        "json_schema": {
          "name": "reasoning_schema",
          "strict": True,
          "schema": {
            "type": "object",
            "properties": {
              "reasoning_steps": {
                "type": "array",
                "items": {
                  "type": "string"
                },
                "description": "The reasoning steps leading to the final conclusion."
              },
              "answer": {
                "type": "string",
                "description": "The final answer, taking into account the reasoning steps."
              }
            },
            "required": ["reasoning_steps", "answer"],
            "additionalProperties": False
          }
        }
      }
    
)
message = completion.choices[0].message


In [127]:
print(json.dumps(json.loads(message.content), indent=4))

{
    "reasoning_steps": [
        "Compare the whole number part of each decimal: 9 and 9. They are equal.",
        "Move to the tenths place: 9.11 has a 1, and 9.9 has a 9. Clearly, the tenths digit of 9.9 is larger than that of 9.11.",
        "However, 9.9 is equivalent to 9.90 when expressed with two decimal places for comparison.",
        "In 9.90: the whole part is 9, the tenths digit is 9, and the hundredths digit is 0.",
        "In 9.11: the whole part is 9, the tenths digit is 1, and the hundredths digit is 1.",
        "Compare the tenths digits considering them with the next significant digit if necessary: compare 9.90 (or use a number line for precise placement) with 9.11.",
        "9.90 has larger value than 9.11 because in decimal numbers with equal whole parts, the comparison reduces down to the value of the tenths and hundredths parts."
    ],
    "answer": "9.9 is bigger than 9.11."
}


In [45]:
meeting = '''Attendees:

Alex (Project Manager)
Jamie (Backend Developer)
Sam (Frontend Developer)
Taylor (UI/UX Designer)
Chris (QA Engineer)
Alex: Good morning, everyone. Thanks for joining the meeting. Let's start with a quick status update on the new feature development. Jamie, how's the backend integration going?

Jamie: Morning, Alex. The backend integration is progressing well. We’ve set up the new API endpoints and they’re working as expected. I’m currently working on optimizing the database queries. I should have it wrapped up by the end of this week.

Alex: That sounds good. Sam, how are things on the frontend side?

Sam: Hi, Alex. The frontend is shaping up nicely. I’ve integrated the API endpoints that Jamie set up, and the new UI components are responsive. There are a few minor bugs I’m working on fixing, but overall, we’re on track.

Alex: Great to hear. Taylor, any updates from the UI/UX perspective?

Taylor: Yes, Alex. We’ve received some feedback from our user testing sessions. Most users found the new feature intuitive, but there were a few suggestions for improving the navigation flow. I’m working on refining the design based on this feedback. I’ll share the updated mockups by tomorrow.

Alex: Excellent. Chris, what about QA? Any major issues we should be aware of?

Chris: Hi, everyone. We’ve started preliminary testing on the new feature. So far, we’ve found a few minor issues related to data validation, but nothing critical. We’ll continue testing and will provide a detailed report by the end of the week.

Alex: Good to know, Chris. Thanks for the updates, everyone. It sounds like we’re making good progress. Before we wrap up, are there any blockers or concerns that we need to address?

Jamie: Just one thing from my side. I might need some help with the performance optimization of the database queries. They’re a bit more complex than I anticipated.

Alex: Noted, Jamie. Sam, can you spare some time to assist Jamie with this?

Sam: Sure, Alex. I can help Jamie with the database optimizations.

Alex: Perfect. Anything else?

Taylor: I just need to confirm the final color scheme for the new UI components. Can we have a quick discussion about that after this meeting?

Alex: Absolutely, Taylor. We can discuss it right after. If there’s nothing else, let’s aim to have another update meeting next week, same time. Thanks, everyone. Keep up the good work.

All: Thank you, Alex.

Post-Meeting Discussion:

Taylor: Alex, regarding the color scheme, I’ve noticed that the current palette might not be as accessible as we’d like. We should consider higher contrast options.

Alex: That’s a good point, Taylor. Let's review the options you have and make sure we adhere to accessibility standards. Can you present a few alternatives?

Taylor: Sure, I’ll prepare a few alternatives and share them with the team for feedback.

Alex: Sounds good. Let’s aim to finalize this by the end of the day. Thanks, Taylor.

Taylor: Will do. Thanks, Alex.'''

In [117]:


client = OpenAI()



completion = client.beta.chat.completions.parse(
      model= "gpt-4o-2024-08-06",
      messages= [
    {
      "role": "system",
      "content": "Extract action items, due dates, and owners from meeting notes."
    },
    {
      "role": "user",
      "content": meeting
    }
  ],
  response_format = {
    "type": "json_schema",
    "json_schema": {
      "name": "action_items",
      "strict": True,
      "schema": {
        "type": "object",
        "properties": {
          "action_items": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "description": {
                  "type": "string",
                  "description": "Description of the action item."
                },
                "due_date": {
                  "type": ["string", "null"],
                  "description": "Due date for the action item, can be null if not specified."
                },
                "owner": {
                  "type": ["string", "null"],
                  "description": "Owner responsible for the action item, can be null if not specified."
                }
              },
              "required": ["description", "due_date", "owner"],
              "additionalProperties": False
            },
            "description": "List of action items from the meeting."
          }
        },
        "required": ["action_items"],
        "additionalProperties": False
      }
    }
  }
)
message = completion.choices[0].message


In [118]:
message.content

'{"action_items":[{"description":"Complete backend database query optimizations.","due_date":"end of this week","owner":"Jamie"},{"description":"Fix minor bugs in the frontend and maintain integration progress.","due_date":null,"owner":"Sam"},{"description":"Refine UI design based on user feedback and share updated mockups.","due_date":"tomorrow","owner":"Taylor"},{"description":"Continue testing new feature and provide a detailed QA report.","due_date":"end of the week","owner":"Chris"},{"description":"Assist Jamie with database performance optimization.","due_date":null,"owner":"Sam"},{"description":"Review and confirm final color scheme for the new UI components using higher contrast options for accessibility.","due_date":"end of the day","owner":"Taylor"}]}'

In [125]:
print(json.dumps(json.loads(message.content), indent=4))

{
    "action_items": [
        {
            "description": "Complete backend database query optimizations.",
            "due_date": "end of this week",
            "owner": "Jamie"
        },
        {
            "description": "Fix minor bugs in the frontend and maintain integration progress.",
            "due_date": null,
            "owner": "Sam"
        },
        {
            "description": "Refine UI design based on user feedback and share updated mockups.",
            "due_date": "tomorrow",
            "owner": "Taylor"
        },
        {
            "description": "Continue testing new feature and provide a detailed QA report.",
            "due_date": "end of the week",
            "owner": "Chris"
        },
        {
            "description": "Assist Jamie with database performance optimization.",
            "due_date": null,
            "owner": "Sam"
        },
        {
            "description": "Review and confirm final color scheme for the new UI component